# The deAlmeida Overland Flow Component 

This notebook illustrates running the deAlmeida overland flow component in an extremely simple-minded way on a real topography, then shows it creating a flood sequence along an inclined surface with an oscillating water surface at one end.

First, import what we'll need:

In [ ]:
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from landlab.components.overland_flow import OverlandFlow
from landlab.io import esri_ascii

Pick the initial and run conditions

In [ ]:
run_time = 100  # duration of run, (s)
h_init = 0.1  # initial thin layer of water (m)
n = 0.01  # roughness coefficient, (s/m^(1/3))
g = 9.8  # gravity (m/s^2)
alpha = 0.7  # time-step factor (nondimensional; from Bates et al., 2010)
u = 0.4  # constant velocity (m/s, de Almeida et al., 2012)
run_time_slices = (10, 50, 100)

Elapsed time starts at 1 second. This prevents errors when setting our boundary conditions.

In [ ]:
elapsed_time = 1.0

Use Landlab methods to import an ARC ascii grid, and load the data into the field that the component needs to look at to get the data. This loads the elevation data, z, into a "field" in the grid itself, defined on the nodes.

In [ ]:
with open("Square_TestBasin.asc") as fp:
    rmg = esri_ascii.load(fp, name="topographic__elevation", at="node")
rmg.set_closed_boundaries_at_grid_edges(True, True, True, True)

We can get at this data with this syntax:

In [ ]:
rmg.at_node["topographic__elevation"]

Note that the boundary conditions for this grid mainly got handled with the final line of those three, but for the sake of completeness, we should probably manually "open" the outlet. We can find and set the outlet like this:

In [ ]:
# This DEM was generated using Landlab and the outlet node ID was known
my_outlet_node = 100
rmg.status_at_node[my_outlet_node] = 1  # 1 is the code for fixed value

Now initialize a couple more grid fields that the component is going to need:

In [ ]:
rmg.add_zeros("surface_water__depth", at="node")  # water depth (m)

In [ ]:
rmg.at_node["surface_water__depth"] += h_init

Let's look at our watershed topography

In [ ]:
rmg.imshow("topographic__elevation")

Now instantiate the component itself

In [ ]:
# for stability in steeper environments, we set the steep_slopes flag to True
of = OverlandFlow(rmg, steep_slopes=True)

Now we're going to run the loop that drives the component:

In [ ]:
with tqdm(total=run_time) as pbar:
    while elapsed_time < run_time:
        # First, we calculate our time step.
        dt = of.calc_time_step()

        # Now, we can generate overland flow.
        of.overland_flow()

        # Increased elapsed time
        elapsed_time += dt

        pbar.update(dt)

In [ ]:
rmg.imshow("surface_water__depth", cmap="Blues")

Now let's get clever, and run a set of time slices:

In [ ]:
elapsed_time = 1.0
for t in run_time_slices:
    while elapsed_time < t:
        # First, we calculate our time step.
        dt = of.calc_time_step()
        # Now, we can generate overland flow.
        of.overland_flow()
        # Increased elapsed time
        elapsed_time += dt
    plt.figure(t)
    rmg.imshow("surface_water__depth", cmap="Blues")